<a href="https://colab.research.google.com/github/kamran8867/Language_Classification/blob/main/Copy_of_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

df = pd.read_csv("Language_Detection.csv")

import re

df['Text'] = df['Text'].str.replace(r'\s*\[\d+\]\s*', ' ', regex=True)
df['Text'] = df['Text'].str.strip().str.replace(r'\s+', ' ', regex=True)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['label'] = le.fit_transform(df['Language'])

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['Text'].tolist(), df['label'].tolist(), test_size=0.2, stratify=df['label'], random_state=42
)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(input_ids=tf.constant(train_encodings['input_ids']),
         attention_mask=tf.constant(train_encodings['attention_mask'])),
    tf.constant(train_labels)
)).shuffle(1000).batch(16)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(input_ids=tf.constant(val_encodings['input_ids']),
         attention_mask=tf.constant(val_encodings['attention_mask'])),
    tf.constant(val_labels)
)).batch(16)

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(le.classes_))

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

model.fit(train_dataset, validation_data=val_dataset, epochs=3)


Saving Language_Detection.csv to Language_Detection.csv


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
517/517 [==============================] - 273s 447ms/step - loss: 0.4396 - accuracy: 0.8954 - val_loss: 0.1630 - val_accuracy: 0.9444
Epoch 2/3
517/517 [==============================] - 227s 439ms/step - loss: 0.1497 - accuracy: 0.9447 - val_loss: 0.2084 - val_accuracy: 0.9309
Epoch 3/3
517/517 [==============================] - 228s 440ms/step - loss: 0.1322 - accuracy: 0.9470 - val_loss: 0.2017 - val_accuracy: 0.9323


In [ ]:

example_text = "Come stai, amico mio?"

inputs = tokenizer(example_text, return_tensors="tf", truncation=True, padding=True, max_length=128)

outputs = model(inputs)
logits = outputs.logits

probs = tf.nn.softmax(logits, axis=1)

predicted_label_idx = tf.argmax(probs, axis=1).numpy()[0]

predicted_language = le.inverse_transform([predicted_label_idx])[0]

print(f"Predicted Language: {predicted_language}")


Predicted Language: Italian
